In [1]:
import pandas as pd
import os
from utils.ml_utils import EmissionsPredictionPipeline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
SCRIPT_DIR_PATH = os.getcwd()
ROOT_DIR_PATH = os.path.dirname(SCRIPT_DIR_PATH)
DATA_DIR_PATH = os.path.join(ROOT_DIR_PATH, "data")
MAPPING_DIR_PATH = os.path.join(DATA_DIR_PATH, "mapping")
PREDICTORS_DIR_PATH = os.path.join(DATA_DIR_PATH, "predictors")
SSP_DIR_PATH = os.path.join(DATA_DIR_PATH, "ssp")
SIMULATION_DIR_PATH = os.path.join(SSP_DIR_PATH, "sisepuede_run_2025-01-14T17;04;06.975301_output_database")
TRAINING_DIR_PATH = os.path.join(DATA_DIR_PATH, "training")

In [4]:
TRAINING_DATA_PATH = os.path.join(TRAINING_DIR_PATH, "training_data.csv")
epp = EmissionsPredictionPipeline(TRAINING_DATA_PATH)

In [5]:
epp.load_data()
epp.df

,1,2,3,4,5,6,7,8,9,10,...,1472,1473,1474,1475,1476,1477,1478,1480,1481,total_emissions_last_five_years
0,0.980563,0.622679,0.140236,0.802093,0.809239,0.204801,0.524637,0.630477,0.832595,0.820676,...,0.194148,0.302870,0.936163,0.489698,0.355706,0.973953,0.163635,0.502731,0.215820,670.507643
1,0.735548,0.301887,0.427054,0.403958,0.065751,0.112043,0.129692,0.464848,0.471658,0.077887,...,0.391113,0.627638,0.037490,0.116765,0.956556,0.605262,0.993768,0.606901,0.649376,604.450538
2,0.483432,0.687238,0.788480,0.682344,0.550297,0.173999,0.258569,0.959298,0.102355,0.432940,...,0.085676,0.160749,0.503487,0.717740,0.322667,0.269358,0.440815,0.146352,0.294294,475.708631
3,0.978340,0.251633,0.057635,0.432074,0.356407,0.050955,0.819088,0.708194,0.664282,0.536167,...,0.671211,0.635634,0.696036,0.291799,0.842492,0.965798,0.610868,0.941222,0.623498,758.004335
4,0.096028,0.866167,0.248441,0.529266,0.495278,0.261648,0.640538,0.341921,0.513636,0.236388,...,0.156381,0.243530,0.098165,0.546683,0.467180,0.449347,0.442861,0.823770,0.297142,600.771431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.686931,0.293175,0.530016,0.537532,0.547453,0.592252,0.145215,0.595690,0.226719,0.555106,...,0.166374,0.492576,0.852423,0.576864,0.581870,0.729968,0.670905,0.526597,0.538860,623.338641
996,0.652014,0.614792,0.722367,0.728350,0.258821,0.451143,0.933635,0.699838,0.318238,0.118908,...,0.081739,0.427041,0.841771,0.700368,0.034557,0.110096,0.358843,0.228176,0.567834,747.358673
997,0.194568,0.471448,0.336939,0.843803,0.291440,0.503017,0.002373,0.607189,0.445903,0.462059,...,0.483941,0.270658,0.453539,0.533885,0.448723,0.985748,0.874981,0.341813,0.518431,634.154137
998,0.298093,0.985266,0.876711,0.357384,0.324181,0.318030,0.521157,0.057600,0.803029,0.038362,...,0.826473,0.188016,0.836447,0.053303,0.036249,0.291268,0.829999,0.494574,0.087424,915.717778


## EDA

In [ ]:
epp.perform_eda()

Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7ff25fa1c860>
Traceback (most recent call last):
  File "/home/tony-ubuntu/anaconda3/envs/metamodel_env/lib/python3.11/site-packages/matplotlib/transforms.py", line 195, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))

KeyboardInterrupt: 


## ML